### Un tutorial (preliminare ed elementare) su MCP con FastMCP 2.0

Luca Mari, maggio 2025

MCP (_Model Context Protocol_) è un protocollo applicativo client-server "di alto livello", una sorta di "meta-HTTP".  
FastMCP 2.0 è un modulo Python (https://gofastmcp.com/getting-started/welcome) che semplifica la realizzazione di client e server MCP, e perciò -- dato lo scopo solo didattico di questo tutorial -- lo useremo qui, costruendo un server che espone alcuni endpoint per operare su file xlsx ([`xlserver.py`), impiegando alcune funzioni Python (in `client_tools.py`](https://huggingface.co/)) per nascondere ulteriormente i dettagli non specificamente rilevanti, e sperimentando variamente in questo notebook.

Per mantenere il contesto il più semplice possibile:
* useremo un modello di linguaggio di piccole dimensioni e in esecuzione locale, per esempio Gemma3-4b oppure Qwen3-4b mediante [LM Studio](https://lmstudio.ai) (tutti liberamente scaricabili da [Hugging Face](https://huggingface.co)) (non spieghiamo qui come attivare un modello di linguaggio in locale);
* eseguiremo il server MCP direttamente sul sistema operativo locale, invece che in remoto o all'interno di un container.

Insomma, una volta installato quanto occorre, questo tutorial può essere eseguito anche disconnessi da internet. La documentazione e l'interazione saranno in italiano, per mostrare le capacità multilingue dei modelli di linguaggio anche di piccole dimensioni.

### Preliminari

* Installare un interprete Python.
* Creare un ambiente virtuale Python e attivarlo.
* Copiare nell'ambiente virtuale questo notebook (`xlclient.ipynb`) e i due file `xlserver.py` e `client_tools.py`.
* Installare nell'ambiente virtuale i moduli `fastmcp`, `openai` (per l'accesso all'API di OpenAI, anche in locale), e `openpyxl` (per operare su file xlsx).
* Installare quanto serve per rendere accessibile un modello di linguaggio in locale via l'API di OpenAI).

### Attivazione del server

Come in ogni interazione client-server, è necessario prima di tutto che il server sia attivo e raggiungibile. MCP consente interazioni sia via standard io sia via http. Eseguire perciò dalla linea di comando:
* `python xlserver.py stdio`  
oppure per esempio:
* `python xlserver.py http 127.0.0.1 8000`

### Connessione del client al server

In base a come si è eseguito il server, se via stdio o http, si può connettere il client scegliendo l'indirizzo a cui connettersi.

In [3]:
import client_tools

#server_address = "xlserver.py" # per connettersi al server via stdio
server_address = "http://127.0.0.1:8000/mcp" # per connettersi al server via http

MCP_client = await client_tools.connect_to_MCP_server(server_address)
try:
    print(await client_tools.is_connected(MCP_client))
except Exception:
    print(f"Errore nella connessione...")

True


### Interazione del client con il server

Una volta connesso al server, il client può chiedere prima di tutto l'elenco degli strumenti messi a disposizione del server, la cui documentazione è visualizzabile come una lista di schemi JSON.  
Questi strumenti sono funzioni Python, implementate in `xlserver.py` e decorate con `@mcp.tool()`.

In [4]:
MCP_server_tools = await client_tools.get_tool_list(MCP_client)
tool_schemas = client_tools.get_tool_schemas(MCP_server_tools)
tool_schemas

[{'type': 'function',
  'function': {'name': 'create_workbook',
   'description': 'Crea un file xlsx dal nome specificato. ',
   'parameters': {'type': 'object',
    'properties': {'filename': {'description': 'Il nome del file xlsx da creare',
      'title': 'Filename',
      'type': 'string'}},
    'required': ['filename']}}},
 {'type': 'function',
  'function': {'name': 'write_data_in_cell',
   'description': 'Assumendo che il file xlsx specificato già esista e non debba essere creato, scrive il valore o la formula specificati nella cella specificata. ',
   'parameters': {'type': 'object',
    'properties': {'filename': {'description': 'Il nome del file xlsx in cui scrivere i dati',
      'title': 'Filename',
      'type': 'string'},
     'cell': {'description': 'La cella in cui scrivere',
      'title': 'Cell',
      'type': 'string'},
     'data': {'description': 'Il valore o la formula da scrivere nella cella',
      'title': 'Data'}},
    'required': ['filename', 'cell', 'data']}

Questi schemi JSON mostrano quali richieste un client MCP può inviare al server, specificando il nome di uno strumento, dunque di una funzione Python, e gli eventuali parametri da passare. Si possono perciò inviare richieste indipendentemente dall'uso di un modello di linguaggio, come in questo esempio.

In [5]:
function_name = "create_workbook"
function_arguments = {"filename": "prova.xlsx"}
function_name2 = "write_data_in_cell"
function_arguments2 = {"filename": "prova.xlsx", "cell": "A1", "data": 1234}

async with MCP_client:
    client_tools.print_msgs(await client_tools.lowlevel_call_MCP_server(MCP_client, function_name, function_arguments))
    client_tools.print_msgs(await client_tools.lowlevel_call_MCP_server(MCP_client, function_name2, function_arguments2))

'File prova.xlsx creato con successo.'
'Dati scritti nel file prova.xlsx con successo.'


A questo punto, e assumendo che un modello di linguaggio sia raggiungibile,
1. lo si può interpellare, inviandogli richieste in italiano che il modello traduce in richieste in formato JSON, grazie alle capacità di _function calling_ del modello stesso (se si usa Qwen3 si può provare ad aggiungere in fondo alla richiesta lo switch `/no_think` per rendere l'elaborazione più rapida),
2. richieste che il client MCP invia al server MCP,
3. che le esegue e produce il risultato richiesto.

In [6]:
model = await client_tools.connect_to_model() # per connettersi al modello via http con i parametri di default di LM Studio

prompt = "Crea un file Excel chiamato 'prova.xlsx'. /no_think."
model_response = client_tools.do(model, prompt, tool_schemas) # passo 1
client_tools.print_tools([tool_call.function for tool_call in model_response.choices[0].message.tool_calls]) # passo 2 # type: ignore
client_tools.print_msgs(await client_tools.call_MCP_server(MCP_client, model_response)) # passo 3

[Function(arguments='{"filename":"prova.xlsx"}', name='create_workbook')]
'File prova.xlsx creato con successo.'


È interessante che, nonostante le sue piccole dimensioni, il modello di linguaggio è in grado di "comprendere" se una richiesta che ha ricevuto deve essere tradotta in una successione di più chiamate a strumenti.

In [7]:
prompt = "Crea un file Excel chiamato 'prova.xlsx' e scrivi il numero 1234 nella cella A1. /no_think."
model_response = client_tools.do(model, prompt, tool_schemas)
client_tools.print_tools([tool_call.function for tool_call in model_response.choices[0].message.tool_calls]) # type: ignore
client_tools.print_msgs(await client_tools.call_MCP_server(MCP_client, model_response))

[   Function(arguments='{"filename":"prova.xlsx"}', name='create_workbook'),
    Function(arguments='{"filename":"prova.xlsx","cell":"A1","data":1234}', name='write_data_in_cell')]
'File prova.xlsx creato con successo.'
'Dati scritti nel file prova.xlsx con successo.'


Dato questo principio, quanto complesse possono essere le richieste traducibili in successioni di chiamate a strumenti diventa una questione di quanto sofisticato è il modello di linguaggio che si sta usando.

In [8]:
prompt = """Crea il file 'prova.xlsx', nelle celle da A1 ad A5 scrivi i numeri interi da 1 a 5,
e nelle celle da B1 a B5 scrivi la formula per calcolare il quadrato del numero
nella cella nella stessa riga della colonna A. /no_think."""
model_response = client_tools.do(model, prompt, tool_schemas)
client_tools.print_tools([tool_call.function for tool_call in model_response.choices[0].message.tool_calls]) # type: ignore
client_tools.print_msgs(await client_tools.call_MCP_server(MCP_client, model_response))

[   Function(arguments='{"filename":"prova.xlsx"}', name='create_workbook'),
    Function(arguments='{"filename":"prova.xlsx","first_cell":"A1","last_cell":"A5","data":[1,2,3,4,5]}', name='write_data_in_range'),
    Function(arguments='{"filename":"prova.xlsx","first_cell":"B1","last_cell":"B5","data":["=A1^2","=A2^2","=A3^2","=A4^2","=A5^2"]}', name='write_data_in_range')]
'File prova.xlsx creato con successo.'
'Dati scritti nel file prova.xlsx con successo.'
'Dati scritti nel file prova.xlsx con successo.'


In [9]:
prompt = """Crea il file 'prova.xlsx' e nelle celle da A1 a C5
scrivi la formula per generare numeri casuali decimali tra 10 e 20. /no_think"""
model_response = client_tools.do(model, prompt, tool_schemas)
client_tools.print_tools([tool_call.function for tool_call in model_response.choices[0].message.tool_calls]) # type: ignore
client_tools.print_msgs(await client_tools.call_MCP_server(MCP_client, model_response))

[   Function(arguments='{"filename":"prova.xlsx"}', name='create_workbook'),
    Function(arguments='{"filename":"prova.xlsx","first_cell":"A1","last_cell":"C5","data":[["=RAND()*(20-10)+10","=RAND()*(20-10)+10","=RAND()*(20-10)+10"],["=RAND()*(20-10)+10","=RAND()*(20-10)+10","=RAND()*(20-10)+10"],["=RAND()*(20-10)+10","=RAND()*(20-10)+10","=RAND()*(20-10)+10"],["=RAND()*(20-10)+10","=RAND()*(20-10)+10","=RAND()*(20-10)+10"],["=RAND()*(20-10)+10","=RAND()*(20-10)+10","=RAND()*(20-10)+10"]]}', name='write_data_in_range')]
'File prova.xlsx creato con successo.'
'Dati scritti nel file prova.xlsx con successo.'
